In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install impyute

In [5]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
import lightgbm as lgb
warnings.filterwarnings(action='ignore')
from impyute.imputation.cs import mice
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [6]:
train_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_problem_data.csv')
test_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_err_data.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/data/sample_submission.csv')
train_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_quality_data.csv')
test_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_quality_data.csv')

In [7]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [8]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

In [9]:
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
  error[person_idx-train_user_id_min,err-1] +=1
error

100%|██████████| 16554663/16554663 [00:46<00:00, 357684.72it/s]


array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [10]:
problem = np.zeros(15000)
problem[train_problem['user_id'].unique()-train_user_id_min] = 1
problem

array([0., 1., 0., ..., 1., 1., 0.])

In [11]:
train_err_df = pd.DataFrame(error)
train_err_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


In [13]:
x = StandardScaler().fit_transform(train_err_df.values)
features = list(range(0,42,1))
pd.DataFrame(x,columns=features).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,-0.196898,-0.030123,0.084345,0.009314,-0.279792,-0.075599,-0.073495,-0.046816,-0.037669,-0.004481,-0.056984,-0.055499,-0.009039,-0.027536,-0.683303,-0.345701,-0.279353,-0.225162,-0.114028,-0.252677,-0.108244,-0.225719,-0.189777,-0.071953,-0.075181,-0.163113,-0.032281,-0.032490,0.0,-0.154519,-0.381844,-0.198957,-1.053094,-0.063401,-0.074263,-1.030661,-1.026064,-0.114398,-0.080135,-0.602484,-0.297800,-0.644056
1,-0.196898,-0.030123,-0.029070,-0.045879,-0.046368,-0.075599,-0.073495,-0.046816,-0.037669,-0.021047,-0.109085,-0.107411,-0.009039,-0.007292,0.626450,0.693337,-0.279353,4.832761,2.461897,5.213885,2.566639,1.146482,0.749306,0.066167,0.877285,-0.596917,-0.033202,-0.033411,0.0,-0.154519,0.339417,-0.198957,-0.168027,0.040010,-0.074263,0.628662,0.622496,-0.114398,-0.080135,0.795711,2.104322,-0.363005
2,-0.196898,-0.030123,-0.000716,0.024173,-0.275388,-0.043639,-0.073495,-0.046816,-0.037669,-0.018681,-0.077824,-0.076264,-0.009039,-0.051828,-0.782958,-0.485273,-0.279353,-0.225162,-0.114028,-0.252677,-0.108244,-0.225719,-0.189777,-0.016705,-0.075181,-0.466776,-0.031360,-0.031569,0.0,-0.154519,-0.381844,-0.198957,-1.053094,-0.063401,-0.074263,-1.030661,-1.026064,-0.114398,-0.080135,-0.602484,-0.297800,-0.644056
3,-0.196898,-0.030123,-0.029070,-0.045879,-0.270984,-0.075599,-0.073495,-0.046816,-0.037669,-0.021047,-0.119505,-0.128176,-0.026327,-0.068023,-0.782958,-0.826450,-0.188277,-0.225162,-0.114028,-0.252677,-0.108244,-0.174896,-0.166018,-0.071953,-0.075181,1.008157,-0.033202,-0.033411,0.0,-0.154519,-0.194316,-0.198957,-0.345040,-0.063401,-0.074263,0.628662,0.622496,0.641705,-0.080135,-0.392136,-0.254905,-0.644056
4,-0.196898,-0.030123,-0.029070,-0.045348,-0.279792,-0.011680,0.015908,-0.046816,-0.037669,-0.021047,-0.046564,-0.024351,0.025536,-0.047779,0.512559,0.119540,-0.279353,-0.225162,-0.114028,-0.252677,-0.108244,0.028393,-0.040974,-0.071953,-0.075181,-0.119733,-0.033202,-0.033411,0.0,-0.154519,0.125924,-0.198957,0.363013,-0.063401,-0.074263,0.628662,0.622496,-0.114398,-0.080135,-0.552991,-0.297800,-0.081954


In [72]:
pca = PCA(n_components=35)
printcipalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data=printcipalComponents)
principalDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,-1.900356,-0.180681,0.661727,-0.104910,-0.139359,-0.222134,0.527824,-0.398696,-0.208936,0.350461,-0.229256,-0.058889,-0.014576,-0.005459,-0.188257,-0.044884,0.093084,0.163756,0.041239,0.076999,-0.199082,0.008928,-0.080286,-0.044058,0.397994,0.067819,0.126095,-0.159805,-0.119587,0.063714,-0.457631,-0.100182,-0.003408,-0.060427,0.010199
1,5.169480,-0.895393,5.645860,0.008439,0.661135,-0.220848,0.093480,-0.407417,0.639307,-0.402592,0.555651,0.227817,-0.173154,0.103391,0.491911,0.137536,-0.033761,-0.029341,1.296443,0.162823,1.182345,0.125103,0.336879,0.338679,1.328371,0.251280,0.343717,-0.308095,0.019286,2.398396,0.479890,0.033363,0.026529,-0.832355,0.014435
2,-2.059069,-0.253401,0.754751,-0.039573,-0.127136,-0.258041,0.414202,-0.360804,-0.155194,0.296727,-0.202623,-0.038547,-0.014575,-0.010836,-0.170276,-0.032448,0.068171,0.154294,0.025213,0.065373,-0.171107,0.017277,-0.028174,0.022671,0.418698,0.110738,0.163346,-0.184013,-0.012267,0.051534,-0.259896,-0.083512,0.053559,-0.062375,0.030866
3,-0.406911,-0.182769,-0.225277,0.006903,-0.427223,-0.231398,-1.225183,0.394739,-0.545101,0.040837,-0.066590,-0.105390,0.057703,-0.001196,-0.125881,0.048122,-0.084730,0.212467,-0.124729,0.137304,-0.088813,0.042061,-0.007768,0.011691,0.355036,-0.176373,0.216907,0.190932,-0.163343,0.022069,-1.388417,-0.050842,0.033049,0.096597,-0.012614
4,0.249358,-0.053803,-0.575960,-0.076312,-0.185234,-0.045679,-0.531642,-0.192700,-0.106339,-0.426571,0.004410,0.293231,-0.039552,-0.116538,0.175028,-0.078358,-0.333316,0.174130,-0.287455,0.145098,-0.086805,0.013821,-0.059069,-0.064787,0.327919,-0.265033,-0.162911,0.007207,-0.549970,-0.303515,0.243882,0.083830,-0.033557,0.071578,-0.036750


In [73]:
pca.explained_variance_ratio_

array([0.10372063, 0.07610478, 0.07352561, 0.06210993, 0.05191624,
       0.05119222, 0.04102495, 0.0310046 , 0.02672474, 0.02587593,
       0.02515257, 0.02472248, 0.02457164, 0.02422904, 0.0241223 ,
       0.02393213, 0.02381464, 0.02290001, 0.02274434, 0.02212234,
       0.02202927, 0.02192162, 0.02170203, 0.02060562, 0.01923957,
       0.01852873, 0.01754507, 0.01702237, 0.01438544, 0.01149707,
       0.00892944, 0.00880045, 0.0076154 , 0.00435637, 0.00171544])

In [74]:
sum(pca.explained_variance_ratio_)

0.9974050161906398

In [75]:
train_x = principalDf.values
train_y = problem
print(train_x.shape)
print(train_y.shape)

(15000, 35)
(15000,)


In [76]:
# dtrain = xgb.DMatrix(data=train_x,label=train_y)

In [77]:
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 50
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

Training until validation scores don't improve for 50 rounds.
[20]	valid_0's auc: 0.783532	valid_0's pr_auc: 0.775152
[40]	valid_0's auc: 0.786987	valid_0's pr_auc: 0.777936
[60]	valid_0's auc: 0.787491	valid_0's pr_auc: 0.779583
[80]	valid_0's auc: 0.786829	valid_0's pr_auc: 0.778687
Early stopping, best iteration is:
[35]	valid_0's auc: 0.787524	valid_0's pr_auc: 0.778634
Training until validation scores don't improve for 50 rounds.
[20]	valid_0's auc: 0.792444	valid_0's pr_auc: 0.338502
[40]	valid_0's auc: 0.795289	valid_0's pr_auc: 0.377945
[60]	valid_0's auc: 0.79084	valid_0's pr_auc: 0.36369
Early stopping, best iteration is:
[28]	valid_0's auc: 0.795654	valid_0's pr_auc: 0.365128
Training until validation scores don't improve for 50 rounds.
[20]	valid_0's auc: 0.760268	valid_0's pr_auc: 0.359205
[40]	valid_0's auc: 0.757692	valid_0's pr_auc: 0.358495
[60]	valid_0's auc: 0.754636	valid_0's pr_auc: 0.362256
Early stopping, best iteration is:
[19]	valid_0's auc: 0.76067	valid_0's p

In [78]:
# 교차검증 점수 확인
print(np.mean(auc_scores))

0.7873509132340919


In [79]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [80]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
  test_x[person_idx-test_user_id_min,err-1]+=1
test_x = test_x.reshape(test_x.shape[0],-1)
test_x.shape

100%|██████████| 16532648/16532648 [00:43<00:00, 376581.46it/s]


(14999, 42)

In [83]:
x = StandardScaler().fit_transform(test_x)
features = list(range(0,42,1))
pd.DataFrame(x,columns=features).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,-0.195744,-0.031265,-0.053584,-0.024235,0.004866,-0.076573,-0.076970,-0.047343,-0.056715,-0.025021,-0.019352,-0.085303,-0.096747,-0.045584,-0.580398,-0.723975,0.217110,2.989508,-0.085127,1.097021,-0.078318,1.435049,1.084125,0.062679,2.92463,-0.156236,-0.034552,-0.034833,0.0,-0.153472,-0.056705,-0.194513,0.346428,-0.065632,0.016874,3.792101,3.765437,-0.133966,0.040200,0.539586,-0.347358,-0.081558
1,-0.195744,-0.031265,0.049534,0.005791,-0.220379,0.006625,0.006359,-0.047343,-0.056715,-0.025021,-0.032687,-0.085303,0.319519,-0.037345,-0.781219,-0.443906,-0.121921,-0.202834,-0.085127,-0.190174,-0.078318,-0.212542,-0.155463,-0.081366,-0.11585,-0.503358,-0.033321,-0.033603,0.0,-0.153472,-0.139871,-0.199937,-1.000969,-0.065632,-0.075328,-1.005008,-1.000353,-0.133966,-0.086726,-0.602543,-0.347358,-0.644892
2,-0.195744,-0.031265,-0.053584,-0.024235,0.069767,-0.048840,-0.076970,-0.047343,-0.056715,-0.025021,-0.019352,-0.085303,-0.096747,-0.051077,0.940100,0.769727,-0.121921,-0.202834,-0.085127,-0.190174,-0.078318,-0.111977,-0.117555,-0.023748,-0.11585,0.190887,-0.034552,-0.034833,0.0,-0.153472,0.090589,-0.199937,0.346428,-0.065632,-0.075328,0.594028,0.588244,-0.133966,0.801754,0.800289,-0.299226,-0.081558
3,-0.195744,-0.031265,-0.053584,-0.024235,0.176663,-0.048840,-0.049194,-0.047343,-0.056715,-0.025021,-0.059357,-0.304713,0.007320,-0.034598,-1.426713,-1.112960,-0.121921,0.861280,-0.085127,1.097021,-0.078318,-0.207513,-0.153568,-0.081366,-0.11585,-1.327774,-0.032090,-0.032372,0.0,-0.153472,-0.005603,-0.199937,-0.664120,0.034604,0.201278,0.594028,0.588244,-0.133966,-0.086726,-0.105965,0.037700,-0.644892
4,-0.195744,-0.031265,-0.053584,-0.023622,-0.208926,-0.076573,-0.076970,-0.047343,-0.056715,-0.025021,-0.039355,-0.165088,0.007320,-0.004387,-0.393922,-0.257193,0.104100,-0.202834,-0.085127,-0.190174,-0.078318,-0.195781,-0.147882,-0.081366,-0.11585,-0.937261,-0.033321,-0.033603,0.0,-0.153472,0.192793,0.212281,-0.074634,-0.028044,0.016874,0.594028,0.588244,0.288700,-0.086726,-0.006650,3.984542,-0.644892


In [84]:
pca = PCA(n_components=35)
printcipalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data=printcipalComponents)
principalDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,3.849484,0.153335,-0.209856,0.031037,-0.360130,0.293771,4.700029,0.407009,0.881568,0.139828,1.266418,-0.759038,-0.038133,-0.880587,0.708024,0.191684,0.234761,-0.672270,-0.832878,0.146657,0.742851,-1.444773,-1.531771,0.548310,0.898495,-0.194993,-0.426529,0.490298,-0.009518,-0.697482,0.154708,-0.063846,-0.096499,-1.249104,-0.017040
1,-1.717129,1.238631,0.254041,-0.047234,-0.251027,-0.014470,-0.454896,-0.147387,-0.303479,-0.107396,-0.063982,0.033821,-0.059135,0.106785,-0.251821,-0.014967,-0.014098,0.062616,0.212785,-0.152884,0.113897,0.114568,-0.243601,0.163958,0.211977,0.222336,-0.240850,-0.001588,0.008846,-0.218803,-0.131505,-0.213451,0.018128,0.022877,0.073729
2,0.841724,-1.153780,-0.354981,0.237730,0.160858,-0.175977,-0.339151,-0.070081,0.135357,-0.327237,0.372660,-0.056774,-0.014019,0.001849,0.251486,0.436450,0.070519,-0.619996,0.194013,-0.151984,0.416165,-0.202837,0.075895,-0.110885,-0.204573,0.433912,-0.279057,0.295074,-0.049164,0.032486,0.025698,0.052717,-0.003277,0.154757,-0.024570
3,-0.304580,1.589680,0.066170,-0.282699,-0.466354,-0.587051,1.562561,-0.122981,0.192786,-0.052452,0.107851,0.120049,-0.456010,-0.192427,0.299037,0.001829,0.037546,0.055782,-0.270480,0.356261,0.319752,0.041686,0.149000,-0.071502,0.366679,0.422534,-0.624367,0.703018,0.197782,-0.023317,0.030793,-0.068154,0.067945,-1.078473,0.005050
4,0.086201,-0.381751,-0.153945,-0.080383,-0.098178,-0.313357,0.688831,-0.192135,0.331141,0.061623,0.181860,0.038320,0.024014,0.160549,0.112972,-0.327614,-0.193491,1.069903,-0.868718,2.949059,-0.658269,1.523070,-0.009077,-0.669264,1.485724,1.104510,-0.335030,0.113928,0.071230,0.214229,0.045569,-0.074922,0.049505,0.209458,0.013321


In [85]:
pca.explained_variance_ratio_

array([0.10551963, 0.08162703, 0.07311925, 0.05974668, 0.0497546 ,
       0.04769242, 0.04078267, 0.03342663, 0.02903789, 0.0272462 ,
       0.0261853 , 0.0257028 , 0.02513293, 0.02475112, 0.02446356,
       0.02419554, 0.02389938, 0.02376061, 0.02334725, 0.02253779,
       0.02163606, 0.0209082 , 0.02022888, 0.01960898, 0.01924298,
       0.0188393 , 0.01766938, 0.01464292, 0.0129419 , 0.00972553,
       0.00829456, 0.00701778, 0.00602711, 0.00467218, 0.00252615])

In [86]:
sum(pca.explained_variance_ratio_)

0.9959111937822308

In [87]:
# 예측
pred_y_list =[]
for model in models:
  pred_y = model.predict(principalDf.values)
  pred_y_list.append(pred_y.reshape(-1,1))
pred_ensemble = np.mean(pred_y_list,axis=0)
pred_ensemble

array([[0.82171917],
       [0.44733971],
       [0.40593259],
       ...,
       [0.5870313 ],
       [0.77768075],
       [0.39675313]])

In [88]:
pred = np.where(pred_ensemble>threshold, 1,0)
pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [ ]:
sample_submission.head(2)

,user_id,problem
0,30000,0
1,30001,0


In [89]:
sample_submission['problem'] = pred
sample_submission.head()

,user_id,problem
0,30000,1
1,30001,0
2,30002,0
3,30003,0
4,30004,0


In [90]:
sample_submission.to_csv('/content/drive/MyDrive/데이콘/submission/lgb_kfold5_only_errordata_pca35_submission_threshold0.5.csv')